In [1]:

import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from train_set import Training_set
from sklearn.model_selection import train_test_split
from numpy import mean, std

In [2]:
chi = pd.read_csv("Chi_data.csv", index_col= 0)
X = chi
#Y has the last col with labels
y = Training_set.iloc[:, -1]
# split the dataset to train and test, the 2nd is for the non feature selection
Training_data = pd.merge(X, y, left_index=True, right_index=True)
print(Training_data)

X = Training_data.iloc[:, :-1]
#Y has the last col with labels
y = Training_data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

           35076296-35282086  41062669-41447005  42161364-42296514  \
Array.129                  2                 -1                 -1   
Array.34                   0                  0                  0   
Array.67                  -1                  0                  0   
Array.24                   0                  1                  1   
Array.22                   0                  0                  0   
...                      ...                ...                ...   
Array.10                   2                 -1                 -1   
Array.123                  0                  0                  0   
Array.100                  0                  0                  0   
Array.134                  1                  1                  1   
Array.130                  2                 -1                 -1   

           85450052-85962613  41458670-41494331  84542006-85443011  \
Array.129                  0                 -1                  0   
Array.34           

In [3]:
# Split the generated data into training and testing sets
skf_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
actual_scores = []

# Define the parameter grid for the random forest classifier
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
for train_index_inner, test_index_inner in skf_outer.split(X_train, y_train):
    X_train_inner, X_test_inner = X_train.iloc[train_index_inner], X_train.iloc[test_index_inner]
    y_train_inner, y_test_inner = y_train.iloc[train_index_inner], y_train.iloc[test_index_inner]

    # parameter tuning for the random forest classifier using GridSearchCV
    rf_model = RandomForestClassifier(random_state=42)

    # Inner cross-validation loop for hyperparameter tuning
    skf_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid, scoring='accuracy', cv=skf_inner)

    grid_search.fit(X_train_inner, y_train_inner)
    best_model = grid_search.best_estimator_
    yhat = best_model.predict(X_test_inner)
    acc = accuracy_score(y_test_inner, yhat)
    actual_scores.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, grid_search.best_score_, grid_search.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(actual_scores), std(actual_scores)))

>acc=0.812, est=0.782, cfg={'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
>acc=0.875, est=0.765, cfg={'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
>acc=0.750, est=0.829, cfg={'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
>acc=0.750, est=0.829, cfg={'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
>acc=0.812, est=0.828, cfg={'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Accuracy: 0.800 (0.047)
